# 🔬 Grafo V2: Prompts Divididos

Este notebook implementa **una versión avanzada del grafo** que divide el prompt monolítico en múltiples nodos especializados.

## 🎯 Objetivo

Tener **máxima observabilidad y control** dividiendo el proceso de codificación en:

1. **Validar** → Filtrar respuestas inválidas
2. **Buscar en Catálogo** → Match con códigos históricos
3. **Generar Nuevos** → Crear códigos para lo no cubierto
4. **Justificar** → Explicar las decisiones
5. **Ensamblar** → Combinar resultados

---

## 📊 Comparación V1 vs V2

| Aspecto | V1 (Simple) | V2 (Dividido) |
|---------|-------------|---------------|
| **Llamadas GPT/batch** | 1 | 4 |
| **Observabilidad** | ❌ Baja | ✅ Alta |
| **Modificabilidad** | ⚠️ Media | ✅ Alta |
| **Costo** | ✅ Menor | ⚠️ Mayor |
| **Control granular** | ❌ No | ✅ Sí |

---

## ⚠️ Cuándo Usar V2

- ✅ Necesitas entender **exactamente** qué hace cada paso
- ✅ Quieres **ajustar** solo una parte del proceso
- ✅ Estás **experimentando** con diferentes estrategias
- ✅ Necesitas **auditoría** detallada

---


## 📋 Prerrequisitos

Este notebook **extiende** el notebook 03. Puedes:

1. **Opción A:** Ejecutar las celdas 1-7 del notebook 03 primero
2. **Opción B:** Copiar las celdas de setup aquí

**Necesitas:**
- ✅ Respuestas cargadas (`respuestas_reales`)
- ✅ Catálogo histórico (`catalogo_historico`)
- ✅ Configuración (`MODELO_GPT`, `BATCH_SIZE`, etc.)

---


In [ ]:
# ========================================
# 📦 REUTILIZAR SETUP DEL NOTEBOOK 03
# ========================================

# Opción 1: Ejecutar el notebook 03 completo
%run 03_experimentacion_real.ipynb

print("✅ Setup del notebook 03 cargado")
print("✅ Variables disponibles: respuestas_reales, catalogo_historico, etc.")


---

## 🔬 Paso 1: Definir Esquemas Pydantic Especializados

Cada nodo tiene su propio esquema de salida:

---


In [ ]:
# ========================================
# 📦 ESQUEMAS PYDANTIC V2
# ========================================

from pydantic import BaseModel, Field
from typing import List

# 1️⃣ Validación
class ValidacionRespuesta(BaseModel):
    respuesta_id: int = Field(description="ID de la respuesta (1-based)")
    es_valida: bool = Field(description="True si la respuesta tiene contenido útil")
    razon: str = Field(description="Breve razón de por qué es válida o no")

class ResultadoValidacion(BaseModel):
    validaciones: List[ValidacionRespuesta]

# 2️⃣ Búsqueda en Catálogo
class CodigoHistoricoMatch(BaseModel):
    codigo: int = Field(description="Código del catálogo histórico")
    relevancia: float = Field(description="Qué tan relevante es (0.0-1.0)", ge=0.0, le=1.0)

class BusquedaCatalogo(BaseModel):
    respuesta_id: int
    codigos_historicos: List[CodigoHistoricoMatch] = Field(
        default_factory=list,
        description="Códigos históricos que aplican"
    )

class ResultadoBusqueda(BaseModel):
    busquedas: List[BusquedaCatalogo]

# 3️⃣ Generación de Nuevos Códigos
class CodigoNuevo(BaseModel):
    descripcion: str = Field(description="Descripción específica del concepto")

class GeneracionNuevos(BaseModel):
    respuesta_id: int
    codigos_nuevos: List[CodigoNuevo] = Field(
        default_factory=list,
        description="Códigos nuevos a crear"
    )

class ResultadoGeneracion(BaseModel):
    generaciones: List[GeneracionNuevos]

# 4️⃣ Justificación
class Justificacion(BaseModel):
    respuesta_id: int
    justificacion: str = Field(description="Explicación de la codificación")

class ResultadoJustificacion(BaseModel):
    justificaciones: List[Justificacion]

print("✅ Esquemas Pydantic V2 definidos")
print(f"   • ValidacionRespuesta")
print(f"   • BusquedaCatalogo")
print(f"   • GeneracionNuevos")
print(f"   • Justificacion")


---

## 🏗️ Paso 2: Definir Nodos del Grafo V2

Implementación de los 5 nodos especializados:

---


In [ ]:
# ========================================
# 🏗️ NODOS DEL GRAFO V2
# ========================================

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# NOTA: Reutilizamos nodos del notebook 03 que no cambian:
# - nodo_inicializar
# - nodo_preparar_batch  
# - nodo_incrementar_batch
# - nodo_normalizar
# - nodo_finalizar
# - decidir_continuar

# Solo definimos los nodos NUEVOS especializados:

def nodo_validar_respuestas_v2(state: EstadoCodificacion) -> EstadoCodificacion:
    """Paso 1: Valida qué respuestas son útiles"""
    print(f"\n✅ Validando {len(state['batch_respuestas'])} respuestas...")
    
    respuestas_str = "\n".join([
        f"{i+1}. {r['texto']}"
        for i, r in enumerate(state["batch_respuestas"])
    ])
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Eres un experto en filtrar respuestas de encuestas.

RECHAZAR si:
- Está vacía o solo tiene "-" o "."
- Es incomprensible
- No responde a la pregunta

ACEPTAR si tiene contenido relevante.

Responde en formato JSON."""),
        ("user", "PREGUNTA: {pregunta}\n\nRESPUESTAS:\n{respuestas}")
    ])
    
    llm = ChatOpenAI(model=state["modelo_gpt"], temperature=0)
    chain = prompt | llm.with_structured_output(ResultadoValidacion)
    
    resultado = chain.invoke({
        "pregunta": state["pregunta"],
        "respuestas": respuestas_str
    })
    
    validas = sum(1 for v in resultado.validaciones if v.es_valida)
    print(f"   ✅ Válidas: {validas}/{len(resultado.validaciones)}")
    
    return {
        **state,
        "validaciones_batch": [v.model_dump() for v in resultado.validaciones]
    }


def nodo_buscar_en_catalogo_v2(state: EstadoCodificacion) -> EstadoCodificacion:
    """Paso 2: Busca códigos históricos relevantes"""
    
    # Solo procesar las válidas
    validas = [
        (i, resp) 
        for i, (resp, val) in enumerate(zip(
            state["batch_respuestas"],
            state["validaciones_batch"]
        ))
        if val["es_valida"]
    ]
    
    if not validas or not state["catalogo"]:
        print(f"   ⚠️  Sin respuestas válidas o sin catálogo")
        return {**state, "busquedas_batch": []}
    
    print(f"\n🔍 Buscando códigos históricos para {len(validas)} respuestas...")
    
    respuestas_validas_str = "\n".join([
        f"{idx+1}. {resp['texto']}"
        for idx, resp in validas
    ])
    
    catalogo_str = "\n".join([
        f"- Código {c['codigo']}: {c['descripcion']}"
        for c in state["catalogo"][:30]
    ])
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""Eres un experto en matching semántico.

CATÁLOGO:
{catalogo_str}

Identifica códigos del catálogo relevantes para cada respuesta.
- Relevancia de 0.0 a 1.0
- Solo incluye si relevancia >= 0.7
- Puedes asignar múltiples códigos

Responde en JSON."""),
        ("user", "PREGUNTA: {pregunta}\n\nRESPUESTAS:\n{respuestas}")
    ])
    
    llm = ChatOpenAI(model=state["modelo_gpt"], temperature=0)
    chain = prompt | llm.with_structured_output(ResultadoBusqueda)
    
    resultado = chain.invoke({
        "pregunta": state["pregunta"],
        "respuestas": respuestas_validas_str
    })
    
    total_matches = sum(len(b.codigos_historicos) for b in resultado.busquedas)
    print(f"   ✅ Matches encontrados: {total_matches}")
    
    return {
        **state,
        "busquedas_batch": [b.model_dump() for b in resultado.busquedas]
    }


def nodo_generar_nuevos_v2(state: EstadoCodificacion) -> EstadoCodificacion:
    """Paso 3: Genera códigos nuevos para lo no cubierto"""
    
    respuestas_sin_match = []
    
    for i, (resp, val) in enumerate(zip(
        state["batch_respuestas"],
        state["validaciones_batch"]
    )):
        if not val["es_valida"]:
            continue
        
        busqueda = next(
            (b for b in state["busquedas_batch"] if b["respuesta_id"] == i+1),
            None
        )
        
        if not busqueda or len(busqueda["codigos_historicos"]) == 0:
            respuestas_sin_match.append((i+1, resp["texto"]))
    
    if not respuestas_sin_match:
        print(f"   ✅ Todas cubiertas con códigos históricos")
        return {**state, "generaciones_batch": []}
    
    print(f"\n🆕 Generando códigos nuevos para {len(respuestas_sin_match)} respuestas...")
    
    respuestas_str = "\n".join([
        f"{resp_id}. {texto}"
        for resp_id, texto in respuestas_sin_match
    ])
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Eres un experto en crear códigos de categorización.

REGLAS:
1. Descripciones ESPECÍFICAS:
   ✅ "Versatilidad de uso"
   ❌ "Versatilidad" (muy genérico)

2. UNIFICA conceptos similares
3. Puedes asignar MÚLTIPLES códigos
4. Sé CONSISTENTE

Responde en JSON."""),
        ("user", "PREGUNTA: {pregunta}\n\nRESPUESTAS:\n{respuestas}")
    ])
    
    llm = ChatOpenAI(model=state["modelo_gpt"], temperature=0)
    chain = prompt | llm.with_structured_output(ResultadoGeneracion)
    
    resultado = chain.invoke({
        "pregunta": state["pregunta"],
        "respuestas": respuestas_str
    })
    
    total_nuevos = sum(len(g.codigos_nuevos) for g in resultado.generaciones)
    print(f"   ✅ Códigos nuevos: {total_nuevos}")
    
    return {
        **state,
        "generaciones_batch": [g.model_dump() for g in resultado.generaciones]
    }


def nodo_justificar_v2(state: EstadoCodificacion) -> EstadoCodificacion:
    """Paso 4: Genera justificaciones"""
    print(f"\n📝 Generando justificaciones...")
    
    resumen_decisiones = []
    
    for i, (resp, val) in enumerate(zip(
        state["batch_respuestas"],
        state["validaciones_batch"]
    )):
        resp_id = i + 1
        
        if not val["es_valida"]:
            resumen_decisiones.append(
                f"{resp_id}. RECHAZADA: {val['razon']}"
            )
            continue
        
        busqueda = next(
            (b for b in state["busquedas_batch"] if b["respuesta_id"] == resp_id),
            {"codigos_historicos": []}
        )
        
        generacion = next(
            (g for g in state["generaciones_batch"] if g["respuesta_id"] == resp_id),
            {"codigos_nuevos": []}
        )
        
        resumen_decisiones.append(
            f"{resp_id}. Hist:{len(busqueda.get('codigos_historicos', []))} "
            f"Nuevos:{len(generacion.get('codigos_nuevos', []))}"
        )
    
    resumen_str = "\n".join(resumen_decisiones)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Genera justificaciones BREVES (1-2 oraciones) explicando:
- Por qué se rechazó
- Por qué se usaron códigos históricos
- Por qué se crearon nuevos

Sé CONCISO. Responde en JSON."""),
        ("user", "DECISIONES:\n{resumen}")
    ])
    
    llm = ChatOpenAI(model=state["modelo_gpt"], temperature=0)
    chain = prompt | llm.with_structured_output(ResultadoJustificacion)
    
    resultado = chain.invoke({"resumen": resumen_str})
    
    print(f"   ✅ Justificaciones generadas")
    
    return {
        **state,
        "justificaciones_batch": [j.model_dump() for j in resultado.justificaciones]
    }


def nodo_ensamblar_resultados_v2(state: EstadoCodificacion) -> EstadoCodificacion:
    """Paso 5: Ensambla resultados finales"""
    print(f"\n🔧 Ensamblando resultados...")
    
    codificaciones_batch = []
    
    for i, (resp, val) in enumerate(zip(
        state["batch_respuestas"],
        state["validaciones_batch"]
    )):
        resp_id = i + 1
        
        justif = next(
            (j for j in state["justificaciones_batch"] if j["respuesta_id"] == resp_id),
            {"justificacion": "Sin justificación"}
        )
        
        if not val["es_valida"]:
            codificaciones_batch.append({
                "fila_excel": resp["fila_excel"],
                "texto": resp["texto"],
                "decision": "rechazar",
                "codigos_historicos": [],
                "codigos_nuevos": [],
                "justificacion": justif["justificacion"]
            })
            continue
        
        busqueda = next(
            (b for b in state["busquedas_batch"] if b["respuesta_id"] == resp_id),
            {"codigos_historicos": []}
        )
        
        generacion = next(
            (g for g in state["generaciones_batch"] if g["respuesta_id"] == resp_id),
            {"codigos_nuevos": []}
        )
        
        codigos_hist = [c["codigo"] for c in busqueda.get("codigos_historicos", [])]
        codigos_nuevos = [c["descripcion"] for c in generacion.get("codigos_nuevos", [])]
        
        if codigos_hist and codigos_nuevos:
            decision = "mixto"
        elif codigos_hist:
            decision = "historico"
        elif codigos_nuevos:
            decision = "nuevo"
        else:
            decision = "rechazar"
        
        codificaciones_batch.append({
            "fila_excel": resp["fila_excel"],
            "texto": resp["texto"],
            "decision": decision,
            "codigos_historicos": codigos_hist,
            "codigos_nuevos": codigos_nuevos,
            "justificacion": justif["justificacion"]
        })
    
    decisiones = {}
    for cod in codificaciones_batch:
        dec = cod["decision"]
        decisiones[dec] = decisiones.get(dec, 0) + 1
    
    print(f"   📊 Decisiones: {decisiones}")
    
    return {
        **state,
        "codificaciones": state["codificaciones"] + codificaciones_batch,
        "mensaje_estado": f"Batch {state['batch_actual'] + 1} completado (V2)"
    }

print("✅ Nodos V2 definidos:")
print("   1. nodo_validar_respuestas_v2")
print("   2. nodo_buscar_en_catalogo_v2")
print("   3. nodo_generar_nuevos_v2")
print("   4. nodo_justificar_v2")
print("   5. nodo_ensamblar_resultados_v2")


---

## 🔗 Paso 3: Construir el Grafo V2

Conectar todos los nodos en el flujo especializado:

---


In [ ]:
# ========================================
# 🔗 CONSTRUIR GRAFO V2
# ========================================

from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Dict

# Extender el estado para incluir campos de V2
class EstadoCodificacionV2(TypedDict):
    # Campos originales (del notebook 03)
    pregunta: str
    modelo_gpt: str
    batch_size: int
    respuestas: List[Dict]
    catalogo: List[Dict]
    batch_actual: int
    total_batches: int
    batch_respuestas: List[Dict]
    codificaciones: List[Dict]
    codigos_nuevos_global: Dict[str, str]
    contador_codigo_nuevo: int
    costo_total: float
    tokens_total: int
    mensaje_estado: str
    progreso_pct: float
    tiempo_inicio: float
    tiempo_total: float
    
    # 🆕 Campos nuevos para V2
    validaciones_batch: List[Dict]
    busquedas_batch: List[Dict]
    generaciones_batch: List[Dict]
    justificaciones_batch: List[Dict]


# Construir el grafo V2
workflow_v2 = StateGraph(EstadoCodificacionV2)

# Agregar nodos (reutilizamos los del notebook 03 cuando sea posible)
workflow_v2.add_node("inicializar", nodo_inicializar)
workflow_v2.add_node("preparar_batch", nodo_preparar_batch)

# 🆕 Nodos especializados de V2
workflow_v2.add_node("validar", nodo_validar_respuestas_v2)
workflow_v2.add_node("buscar_catalogo", nodo_buscar_en_catalogo_v2)
workflow_v2.add_node("generar_nuevos", nodo_generar_nuevos_v2)
workflow_v2.add_node("justificar", nodo_justificar_v2)
workflow_v2.add_node("ensamblar", nodo_ensamblar_resultados_v2)

workflow_v2.add_node("incrementar", nodo_incrementar_batch)
workflow_v2.add_node("normalizar", nodo_normalizar)
workflow_v2.add_node("finalizar", nodo_finalizar)

# Conectar flujo
workflow_v2.set_entry_point("inicializar")
workflow_v2.add_edge("inicializar", "preparar_batch")

# 🔄 Flujo de codificación dividido en 5 pasos
workflow_v2.add_edge("preparar_batch", "validar")
workflow_v2.add_edge("validar", "buscar_catalogo")
workflow_v2.add_edge("buscar_catalogo", "generar_nuevos")
workflow_v2.add_edge("generar_nuevos", "justificar")
workflow_v2.add_edge("justificar", "ensamblar")

# Continuar con siguiente batch o normalizar
workflow_v2.add_edge("ensamblar", "incrementar")

workflow_v2.add_conditional_edges(
    "incrementar",
    decidir_continuar,
    {
        "preparar_batch": "preparar_batch",
        "normalizar": "normalizar"
    }
)

workflow_v2.add_edge("normalizar", "finalizar")
workflow_v2.add_edge("finalizar", END)

# Compilar
app_v2 = workflow_v2.compile()

print("✅ Grafo V2 compilado exitosamente")
print("\n📊 Flujo del Grafo V2:")
print("   inicializar → preparar_batch")
print("   ↓")
print("   validar → buscar_catalogo → generar_nuevos → justificar → ensamblar")
print("   ↓")
print("   incrementar → [preparar_batch ó normalizar]")
print("   ↓")
print("   finalizar")


---

## 📊 Paso 4: Visualizar el Grafo V2

Ahora veamos cómo se ve la estructura del grafo:

---


In [ ]:
# ========================================
# 📊 VISUALIZAR GRAFO V2
# ========================================

from IPython.display import Image, display

print("="*60)
print("📊 VISUALIZACIÓN DEL GRAFO V2")
print("="*60)

# Obtener info del grafo
graph_v2 = app_v2.get_graph()

print(f"\n📋 Información:")
print(f"   Total de nodos: {len(graph_v2.nodes)}")

# Lista de nodos
print(f"\n🔹 Nodos del grafo:")
for i, node_id in enumerate(graph_v2.nodes.keys(), 1):
    print(f"   {i}. {node_id}")

# Vista ASCII
print(f"\n{'='*60}")
print("📊 DIAGRAMA ASCII")
print("="*60)
print(graph_v2.draw_ascii())

# Intentar generar PNG
print(f"\n{'='*60}")
print("🖼️  DIAGRAMA GRÁFICO")
print("="*60)

try:
    img_data = graph_v2.draw_mermaid_png()
    display(Image(img_data))
    print("\n✅ Imagen generada correctamente")
except Exception as e:
    print("⚠️  No se pudo generar imagen PNG")
    print(f"   Razón: {str(e)[:80]}...")
    print("\n📋 Código Mermaid (pega en https://mermaid.live):")
    print("="*60)
    mermaid_code = graph_v2.draw_mermaid()
    print(mermaid_code)
    print("="*60)
    
    # Guardar en archivo
    output_dir = project_root / "notebooks" / "diagramas"
    output_dir.mkdir(exist_ok=True)
    output_file = output_dir / "grafo_v2.mmd"
    output_file.write_text(mermaid_code, encoding="utf-8")
    print(f"\n💾 Código guardado en: notebooks/diagramas/grafo_v2.mmd")
    print("💡 Visita https://mermaid.live y pega el contenido del archivo")


---

## 🆚 Comparar V1 vs V2

Veamos las diferencias estructurales:

---


In [ ]:
# ========================================
# 🆚 COMPARACIÓN V1 vs V2
# ========================================

print("="*60)
print("🆚 COMPARACIÓN: V1 (Simple) vs V2 (Dividido)")
print("="*60)

# Análisis del grafo V1 (del notebook 03)
graph_v1 = app.get_graph()
nodos_v1 = len(graph_v1.nodes)
nodos_logicos_v1 = sum(1 for n in graph_v1.nodes.keys() if not n.startswith("__"))

# Análisis del grafo V2
nodos_v2 = len(graph_v2.nodes)
nodos_logicos_v2 = sum(1 for n in graph_v2.nodes.keys() if not n.startswith("__"))

# Mostrar comparación
print(f"\n{'Métrica':<30} {'V1 (Simple)':<20} {'V2 (Dividido)':<20}")
print("-" * 70)
print(f"{'Total de nodos':<30} {nodos_v1:<20} {nodos_v2:<20}")
print(f"{'Nodos lógicos':<30} {nodos_logicos_v1:<20} {nodos_logicos_v2:<20}")

diferencia = nodos_logicos_v2 - nodos_logicos_v1
porcentaje = (diferencia / nodos_logicos_v1) * 100 if nodos_logicos_v1 > 0 else 0

print(f"\n📊 Diferencia: +{diferencia} nodos lógicos (+{porcentaje:.1f}%)")

# Análisis por batch
print(f"\n{'='*60}")
print("📈 ANÁLISIS DE COMPLEJIDAD POR BATCH")
print("="*60)

print(f"\n🔹 V1 (Simple):")
print(f"   • Nodos ejecutados por batch: ~5")
print(f"   • Llamadas a GPT por batch: 1")
print(f"   • Observabilidad: ⚠️  Baja")

print(f"\n🔹 V2 (Dividido):")
print(f"   • Nodos ejecutados por batch: ~10")
print(f"   • Llamadas a GPT por batch: 4")
print(f"   • Observabilidad: ✅ Alta")

print(f"\n💰 IMPACTO EN COSTOS:")
print(f"   Con 100 respuestas y batch_size=10:")
print(f"   • V1: 10 batches × 1 llamada = 10 llamadas GPT")
print(f"   • V2: 10 batches × 4 llamadas = 40 llamadas GPT (4x más caro)")

print(f"\n✅ RECOMENDACIONES:")
print(f"   • Usa V1 para: Producción, menor costo, velocidad")
print(f"   • Usa V2 para: Experimentación, ajustes finos, auditoría")


---

## 🚀 Paso 5: Ejecutar con Datos Reales

Ahora vamos a procesar tus datos reales con el Grafo V2:

---


In [ ]:
# ========================================
# 🚀 PREPARAR ESTADO INICIAL V2
# ========================================

import time

# Estado inicial extendido con campos de V2
estado_inicial_v2 = {
    "pregunta": nombre_pregunta,
    "modelo_gpt": MODELO_GPT,
    "batch_size": BATCH_SIZE,
    "respuestas": respuestas_reales,
    "catalogo": catalogo_historico,
    "batch_actual": 0,
    "total_batches": 0,
    "batch_respuestas": [],
    "codificaciones": [],
    "codigos_nuevos_global": {},
    "contador_codigo_nuevo": 1,
    "costo_total": 0.0,
    "tokens_total": 0,
    "mensaje_estado": "Iniciando",
    "progreso_pct": 0.0,
    "tiempo_inicio": time.time(),
    "tiempo_total": 0.0,
    
    # 🆕 Campos adicionales para V2
    "validaciones_batch": [],
    "busquedas_batch": [],
    "generaciones_batch": [],
    "justificaciones_batch": []
}

print("="*60)
print("📋 ESTADO INICIAL V2")
print("="*60)

print(f"\n📊 Configuración:")
print(f"   Pregunta: {nombre_pregunta}")
print(f"   Total respuestas: {len(respuestas_reales)}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Catálogo histórico: {len(catalogo_historico)} códigos")
print(f"   Modelo GPT: {MODELO_GPT}")

# Calcular batches esperados
batches_esperados = (len(respuestas_reales) + BATCH_SIZE - 1) // BATCH_SIZE
print(f"\n🧮 Cálculo de batches:")
print(f"   {len(respuestas_reales)} respuestas ÷ {BATCH_SIZE} por batch")
print(f"   = {batches_esperados} batches esperados")

if batches_esperados == 0:
    print(f"\n❌ ERROR: No hay respuestas para procesar!")
else:
    print(f"\n✅ Listo para ejecutar con {batches_esperados} batches")
    
# Estimar llamadas y costo
llamadas_v1 = batches_esperados * 1
llamadas_v2 = batches_esperados * 4

print(f"\n💰 Estimación de llamadas GPT:")
print(f"   V1 (Simple): {llamadas_v1} llamadas")
print(f"   V2 (Dividido): {llamadas_v2} llamadas (4x más)")
print(f"\n⚠️  V2 será aproximadamente 4x más costoso que V1")


In [ ]:
# ========================================
# 🚀 EJECUTAR GRAFO V2 CON DATOS REALES
# ========================================

from langgraph.pregel.main import RunnableConfig

# Calcular límite de recursión apropiado
# V2 tiene más nodos por batch (~10 vs ~5), necesitamos más margen
limite_recursion = max(batches_esperados * 15 + 20, 100)

config_v2 = RunnableConfig(
    recursion_limit=limite_recursion
)

print("="*60)
print("🚀 INICIANDO CODIFICACIÓN CON GRAFO V2")
print("="*60)

print(f"\n⚙️  Configuración de ejecución:")
print(f"   Límite de recursión: {limite_recursion}")
print(f"   (suficiente para {batches_esperados} batches)")

print(f"\n{'='*60}")
print("EJECUTANDO GRAFO V2...")
print("="*60)

try:
    # EJECUTAR
    resultado_final_v2 = app_v2.invoke(estado_inicial_v2, config=config_v2)
    
    print("\n" + "="*60)
    print("✅ PROCESO V2 COMPLETADO EXITOSAMENTE")
    print("="*60)
    
    # Mostrar estadísticas
    print(f"\n📊 Estadísticas finales:")
    print(f"   Tiempo total: {resultado_final_v2['tiempo_total']:.1f}s")
    print(f"   Respuestas procesadas: {len(resultado_final_v2['codificaciones'])}")
    print(f"   Códigos nuevos generados: {len(resultado_final_v2['codigos_nuevos_global'])}")
    
    # Análisis de decisiones
    decisiones_v2 = {}
    for cod in resultado_final_v2["codificaciones"]:
        dec = cod["decision"]
        decisiones_v2[dec] = decisiones_v2.get(dec, 0) + 1
    
    print(f"\n📈 Distribución de decisiones:")
    for decision, cantidad in sorted(decisiones_v2.items(), key=lambda x: -x[1]):
        porcentaje = (cantidad / len(resultado_final_v2['codificaciones'])) * 100
        print(f"   {decision:12} : {cantidad:3} ({porcentaje:.1f}%)")
    
except Exception as e:
    print("\n" + "="*60)
    print("❌ ERROR DURANTE EJECUCIÓN V2")
    print("="*60)
    print(f"\nError: {e}")
    print(f"\nTipo: {type(e).__name__}")
    
    import traceback
    print("\n📋 Traceback completo:")
    traceback.print_exc()
    
    raise


---

## 📊 Paso 6: Analizar Resultados

Veamos qué tan diferente es V2 vs V1:

---


In [ ]:
# ========================================
# 📊 ANÁLISIS COMPARATIVO DE RESULTADOS
# ========================================

print("="*60)
print("📊 ANÁLISIS: Resultados V1 vs V2")
print("="*60)

# Resultados de V1 (del notebook 03)
decisiones_v1 = {}
for cod in resultado_final["codificaciones"]:
    dec = cod["decision"]
    decisiones_v1[dec] = decisiones_v1.get(dec, 0) + 1

print(f"\n🔹 GRAFO V1 (Simple):")
print(f"   Tiempo: {resultado_final['tiempo_total']:.1f}s")
print(f"   Respuestas: {len(resultado_final['codificaciones'])}")
print(f"   Códigos nuevos: {len(resultado_final['codigos_nuevos_global'])}")
print(f"\n   Decisiones:")
for decision, cantidad in sorted(decisiones_v1.items(), key=lambda x: -x[1]):
    porcentaje = (cantidad / len(resultado_final['codificaciones'])) * 100
    print(f"      {decision:12} : {cantidad:3} ({porcentaje:.1f}%)")

print(f"\n🔹 GRAFO V2 (Dividido):")
print(f"   Tiempo: {resultado_final_v2['tiempo_total']:.1f}s")
print(f"   Respuestas: {len(resultado_final_v2['codificaciones'])}")
print(f"   Códigos nuevos: {len(resultado_final_v2['codigos_nuevos_global'])}")
print(f"\n   Decisiones:")
for decision, cantidad in sorted(decisiones_v2.items(), key=lambda x: -x[1]):
    porcentaje = (cantidad / len(resultado_final_v2['codificaciones'])) * 100
    print(f"      {decision:12} : {cantidad:3} ({porcentaje:.1f}%)")

# Comparación
print(f"\n{'='*60}")
print("🆚 DIFERENCIAS")
print("="*60)

diferencia_tiempo = resultado_final_v2['tiempo_total'] - resultado_final['tiempo_total']
diferencia_pct = (diferencia_tiempo / resultado_final['tiempo_total']) * 100 if resultado_final['tiempo_total'] > 0 else 0

print(f"\n⏱️  Tiempo:")
print(f"   V2 tomó {abs(diferencia_tiempo):.1f}s {'más' if diferencia_tiempo > 0 else 'menos'}")
print(f"   ({abs(diferencia_pct):.1f}% {'más lento' if diferencia_tiempo > 0 else 'más rápido'})")

diferencia_codigos = len(resultado_final_v2['codigos_nuevos_global']) - len(resultado_final['codigos_nuevos_global'])
print(f"\n🆕 Códigos nuevos:")
print(f"   V2 generó {abs(diferencia_codigos)} códigos {'más' if diferencia_codigos > 0 else 'menos'} que V1")

print(f"\n💡 OBSERVACIONES:")
if diferencia_tiempo > 0:
    print(f"   • V2 es más lento debido a 4x más llamadas GPT")
else:
    print(f"   • Tiempos similares (las llamadas fueron rápidas)")

if diferencia_codigos != 0:
    print(f"   • V2 generó diferentes códigos (prompts más específicos)")
else:
    print(f"   • Ambos generaron la misma cantidad de códigos")


---

## 📥 Paso 7: Exportar Resultados

Guardar los resultados de V2 para análisis posterior:

---


In [ ]:
# ========================================
# 📥 EXPORTAR RESULTADOS V2
# ========================================

import pandas as pd
from datetime import datetime

# Preparar DataFrame con resultados de V2
datos_exportar_v2 = []

for cod in resultado_final_v2["codificaciones"]:
    # Preparar códigos como strings
    if cod["decision"] == "rechazar":
        codigo_final = "RECHAZADO"
    else:
        codigos = []
        # Códigos históricos
        if cod["codigos_historicos"]:
            codigos.extend([str(c) for c in cod["codigos_historicos"]])
        # Códigos nuevos (buscar sus números)
        if cod["codigos_nuevos"]:
            for desc in cod["codigos_nuevos"]:
                # Buscar el código numérico asignado
                codigo_num = next(
                    (k for k, v in resultado_final_v2["codigos_nuevos_global"].items() if v == desc),
                    desc  # Si no se encuentra, usar la descripción
                )
                codigos.append(str(codigo_num))
        
        codigo_final = "; ".join(codigos) if codigos else "SIN_CODIGO"
    
    datos_exportar_v2.append({
        "Respuesta": cod["texto"],
        "Código": codigo_final,
        "Decisión": cod["decision"],
        "Justificación": cod.get("justificacion", "")
    })

df_resultados_v2 = pd.DataFrame(datos_exportar_v2)

# Generar nombre de archivo
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
nombre_archivo_base = ARCHIVO_RESPUESTAS.stem
modelo_limpio = MODELO_GPT.replace("-", "_").replace(".", "_")

archivo_salida_v2 = project_root / "notebooks" / "resultados" / f"{nombre_archivo_base}_v2_{modelo_limpio}_{timestamp}.xlsx"
archivo_salida_v2.parent.mkdir(exist_ok=True)

# Guardar
df_resultados_v2.to_excel(archivo_salida_v2, index=False)

print("="*60)
print("📥 EXPORTACIÓN COMPLETADA")
print("="*60)

print(f"\n✅ Archivo guardado en:")
print(f"   {archivo_salida_v2.relative_to(project_root)}")

print(f"\n📊 Contenido:")
print(f"   Filas: {len(df_resultados_v2)}")
print(f"   Columnas: {list(df_resultados_v2.columns)}")

print(f"\n📋 Vista previa (primeras 3 filas):")
print(df_resultados_v2.head(3).to_string())

# También guardar catálogo de códigos nuevos si hay
if resultado_final_v2["codigos_nuevos_global"]:
    archivo_catalogo_v2 = project_root / "notebooks" / "resultados" / f"catalogo_nuevos_v2_{timestamp}.xlsx"
    
    df_catalogo_v2 = pd.DataFrame([
        {"Código": cod, "Descripción": desc}
        for cod, desc in resultado_final_v2["codigos_nuevos_global"].items()
    ])
    
    df_catalogo_v2.to_excel(archivo_catalogo_v2, index=False)
    
    print(f"\n✅ Catálogo de códigos nuevos guardado en:")
    print(f"   {archivo_catalogo_v2.relative_to(project_root)}")
    print(f"   Total códigos: {len(df_catalogo_v2)}")


---

## 🔍 Inspección Detallada (Opcional)

Veamos cómo funcionó cada nodo en detalle para un batch específico:

---


In [ ]:
# ========================================
# 🔍 INSPECCIÓN DETALLADA (EJEMPLOS)
# ========================================

print("="*60)
print("🔍 INSPECCIÓN: Ejemplos de Resultados V2")
print("="*60)

# Mostrar algunos ejemplos de cada tipo de decisión
print("\n📋 Ejemplos por tipo de decisión:\n")

for tipo_decision in ["historico", "nuevo", "mixto", "rechazar"]:
    ejemplos = [
        cod for cod in resultado_final_v2["codificaciones"]
        if cod["decision"] == tipo_decision
    ][:2]  # Solo primeros 2 de cada tipo
    
    if ejemplos:
        print(f"\n{'='*60}")
        print(f"🔹 {tipo_decision.upper()}")
        print("="*60)
        
        for i, ej in enumerate(ejemplos, 1):
            print(f"\n{i}. Respuesta: \"{ej['texto'][:80]}...\"")
            
            if ej["decision"] != "rechazar":
                if ej["codigos_historicos"]:
                    print(f"   📚 Códigos históricos: {ej['codigos_historicos']}")
                if ej["codigos_nuevos"]:
                    print(f"   🆕 Códigos nuevos: {ej['codigos_nuevos']}")
            
            if ej.get("justificacion"):
                print(f"   💬 Justificación: {ej['justificacion']}")

# Mostrar códigos nuevos más comunes (si hay)
if resultado_final_v2["codigos_nuevos_global"]:
    print(f"\n{'='*60}")
    print("🆕 CATÁLOGO DE CÓDIGOS NUEVOS GENERADOS")
    print("="*60)
    
    # Contar uso de cada código
    uso_codigos = {}
    for cod in resultado_final_v2["codificaciones"]:
        for desc in cod["codigos_nuevos"]:
            uso_codigos[desc] = uso_codigos.get(desc, 0) + 1
    
    # Top 10 más usados
    top_codigos = sorted(uso_codigos.items(), key=lambda x: -x[1])[:10]
    
    print(f"\n📊 Top 10 códigos nuevos más usados:\n")
    for i, (desc, cantidad) in enumerate(top_codigos, 1):
        codigo_num = next(
            (k for k, v in resultado_final_v2["codigos_nuevos_global"].items() if v == desc),
            "?"
        )
        print(f"{i:2}. [{codigo_num:4}] {desc[:50]:50} (usado {cantidad} veces)")

print(f"\n{'='*60}")
print("✅ Inspección completada")
print("="*60)


---

## ✅ Resumen Final

### **🎯 Qué Lograste:**

1. ✅ Ejecutaste el **Grafo V2** con prompts divididos
2. ✅ Procesaste tus **datos reales**
3. ✅ Comparaste **V1 vs V2** en calidad y tiempo
4. ✅ Exportaste **resultados** a Excel
5. ✅ Analizaste **códigos nuevos** generados

---

### **📊 Decisión: ¿V1 o V2?**

#### **Usa V1 (Notebook 03) si:**
- ✅ Los resultados son suficientemente buenos
- ✅ Quieres **menor costo** (1 llamada vs 4 por batch)
- ✅ Velocidad es importante
- ✅ Vas a **producción**

#### **Usa V2 (Este notebook) si:**
- ✅ Necesitas **ajustar** validación/búsqueda/generación por separado
- ✅ Quieres **máxima observabilidad**
- ✅ Estás **experimentando** con prompts
- ✅ Necesitas **justificaciones** detalladas
- ✅ El costo extra vale la pena

---

### **🚀 Próximos Pasos:**

1. **Revisar resultados** en los archivos Excel generados
2. **Comparar manualmente** 10-20 ejemplos de V1 vs V2
3. **Ajustar prompts** en nodos específicos si es necesario
4. **Decidir arquitectura** para llevar a producción
5. **Migrar código** al backend cuando esté listo

---

### **💡 Tips para Optimizar V2:**

1. **Modelos diferentes por etapa:**
   ```python
   # En validar y generar: usa gpt-4o (preciso)
   # En buscar y justificar: usa gpt-3.5-turbo (barato)
   ```

2. **Ajustar umbrales:**
   ```python
   # En buscar_catalogo_v2:
   # Cambia: relevancia >= 0.7
   # Prueba: relevancia >= 0.8 (más estricto)
   ```

3. **Caché para respuestas similares:**
   - Detecta respuestas casi idénticas
   - Reutiliza codificación previa
   - Reduce llamadas a GPT

---

**¿Preguntas? Revisa `README_NOTEBOOKS.md` en la carpeta notebooks** 📚

---
